<a href="https://colab.research.google.com/github/olonok69/Reinforcement-Learning/blob/master/advanced_saving_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Baselines3 - Advanced Saving and Loading

Github Repo: [https://github.com/DLR-RM/stable-baselines3](https://github.com/DLR-RM/stable-baselines3)


[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) is a collection of pre-trained Reinforcement Learning agents using Stable-Baselines3.

It also provides basic scripts for training, evaluating agents, tuning hyperparameters and recording videos.

Documentation is available online: [https://stable-baselines3.readthedocs.io/](https://stable-baselines3.readthedocs.io/)

## Introduction

In this notebook, you will learn how to use some advanced features of stable baselines3 (SB3): how to easily create a test environment for periodic evaluation, use a policy independently from a model (and how to save it, load it) and save/load a replay buffer.

## Install Dependencies and Stable Baselines Using Pip


```
pip install stable-baselines3[extra]
```

In [1]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 153kB 4.4MB/s 


## Import policy, RL agent, ...

In [2]:
from stable_baselines3 import SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy

## Create the Gym env and instantiate the agent

For this example, we will use Pendulum environment.

"The inverted pendulum swingup problem is a classic problem in the control literature. In this version of the problem, the pendulum starts in a random position, and the goal is to swing it up so it stays upright."

Pendulum-v0 environment: [https://gym.openai.com/envs/Pendulum-v0/](https://gym.openai.com/envs/Pendulum-v0/)

![Pendulum](https://gym.openai.com/videos/2019-10-21--mqt8Qj1mwo/Pendulum-v0/poster.jpg)


We chose the MlpPolicy because input of Pendulum is a feature vector, not images.

The type of action to use (discrete/continuous) will be automatically deduced from the environment action space



### Create the environment and evaluation environment

Stable-Baselines3 allows to automatically create an environment for evaluation.
For that, you only need to specify `create_eval_env=True` when passing the Gym ID of the environment.

In [3]:
model = SAC('MlpPolicy', 'Pendulum-v0', verbose=1, learning_rate=1e-3, create_eval_env=True)

Using cuda device
Creating environment from the given name 'Pendulum-v0'
Creating environment from the given name 'Pendulum-v0'
Wrapping the env in a DummyVecEnv.


Train the agent and evaluate it periodically on the test env.

Behind the scene, SB3 uses an [EvalCallback](https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html#evalcallback)

In [4]:
# Evaluate the model every 1000 steps on 5 test episodes and save the evaluation to the logs folder
model.learn(6000, eval_freq=1000, n_eval_episodes=5, eval_log_path="./logs/")

Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.33e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 81        |
|    time_elapsed    | 9         |
|    total timesteps | 800       |
| train/             |           |
|    actor_loss      | 21        |
|    critic_loss     | 0.12      |
|    ent_coef        | 0.505     |
|    ent_coef_loss   | -1.01     |
|    learning_rate   | 0.001     |
|    n_updates       | 699       |
----------------------------------
Eval num_timesteps=1000, episode_reward=-1733.51 +/- 93.83
Episode length: 200.00 +/- 0.00
New best mean reward!
----------------------------------
| eval/              |           |
|    mean_ep_length  | 200       |
|    mean_reward     | -1.73e+03 |
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.31e+03 |
| time/     

## Save/Load the replay buffer

By default, the replay buffer is not saved when calling `model.save()`, in order to save space on the disk (a replay buffer can be up to several GB when using images).
However, SB3 provides a `save_replay_buffer()` and `load_replay_buffer()` method to save it separately.

In [5]:
# save the model
model.save("sac_pendulum")

# the saved model does not contain the replay buffer
loaded_model = SAC.load("sac_pendulum")
print(f"The loaded_model has {loaded_model.replay_buffer.size()} transitions in its buffer")

The loaded_model has 0 transitions in its buffer


In [6]:
# now save the replay buffer too
model.save_replay_buffer("sac_replay_buffer")

# load it into the loaded_model
loaded_model.load_replay_buffer("sac_replay_buffer")

# now the loaded replay is not empty anymore
print(f"The loaded_model has {loaded_model.replay_buffer.size()} transitions in its buffer")

The loaded_model has 6000 transitions in its buffer


## Save the policy only

In SB3, you save the policy independently from the model if needed.

Note: if you don't save the complete model, you cannot continue training afterward

In [12]:
policy = model.policy
policy.save("sac_policy_pendulum.pkl")

In [13]:
env = model.get_env()

In [14]:
# Evaluate the policy
mean_reward, std_reward = evaluate_policy(policy, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-1291.03 +/- 555.0111083984375


## Load the policy only

In [15]:
from stable_baselines3.sac.policies import MlpPolicy

In [17]:
saved_policy = MlpPolicy.load("/content/sac_policy_pendulum.pkl")

In [18]:
# Evaluate the loaded policy
mean_reward, std_reward = evaluate_policy(saved_policy, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-457.92 +/- 554.127197265625
